In [62]:
import warnings
import os
import requests
import json

from langchain.chat_models.gigachat import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.tools import tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import requests
import json
import datetime
from dateutil import parser
from pydantic import BaseModel, Field
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [63]:
load_dotenv('../.env')

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [64]:
class SimpleChatBot:
    def __init__(self, llm, memory_type, **kwargs):
        self.llm = llm
        self.memory_type = memory_type

        if self.memory_type == "simple":
            self.memory = ConversationBufferMemory()
        elif self.memory_type == "window":
            self.memory = ConversationBufferWindowMemory(k=kwargs["memory_window_size"])
        elif self.memory_type == "summary":
            self.memory = ConversationSummaryMemory(llm=self.llm)
        else:
            raise ValueError("memory_type must be only simple, window or summary!")
        
        self.conversation = ConversationChain(llm=self.llm,
                                              verbose=True,
                                              memory=self.memory)
    
    def print_memory(self):
        if self.memory_type == "simple":
            print("\nSimple memory:")
        elif self.memory_type == "window":
            print("\nWindow memory:")
        elif self.memory_type == "summary":
            print("\nSummary memory:")
        else:
            raise ValueError("memory_type must be only simple, window or summary!")
        print(self.memory.buffer)

    def run(self, user_input):        
        response = self._respond(user_input)

        print(f"AI: {response}")

    def _respond(self, user_input):
        return self.conversation.predict(input=user_input)


Now let's check how it works with each type of memory

In [65]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run("Hello!")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Hello!
AI:

> Finished chain.
AI: Hi there! How can I help you today?

Summary memory:
Human: Hello! How can I help you today?


In [66]:
chat.run("Tell me some interesting facts about beer")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello! How can I help you today?
Human: Tell me some interesting facts about beer
AI:

> Finished chain.
AI: Sure! Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made from fermented grains like barley, wheat, or rice. Today, there are many different types of beer, each with its own unique flavor and ingredients. Some popular beers include Budweiser, Corona, Heineken, Guinness, and Coors Light. Beer is also a major industry, with millions of people working in breweries and related fields around the world.

Summary memory:
Human: Hello! How can I help you today?
AI: Tell me some interesting facts about beer. Beer is the world's oldest alcoholic

In [67]:
chat.run("What kinds of beer are there?")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello! How can I help you today?
AI: Tell me some interesting facts about beer. Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made from fermented grains like barley, wheat, or rice. Today, there are many different types of beer, each with its own unique flavor and ingredients. Some popular beers include Budweiser, Corona, Heineken, Guinness, and Coors Light. Beer is also a major industry, with millions of people working in breweries and related fields around the world.
Human: What kinds of beer are there?
AI:

> Finished chain.
AI: There are many different types of beer, each with its own unique flavor and ingredients. Some popular beers inc

In [68]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run("Hello!")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Hello!
AI:

> Finished chain.
AI: Hi there! How can I help you today?

Window memory:
Human: Hello!
AI: Hi there! How can I help you today?


In [69]:
chat.run("Tell me some interesting facts about beer")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about beer
AI:

> Finished chain.
AI: Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made by fermenting grains like barley, wheat, or rice. The first known recipe for beer was discovered in a Sumerian clay tablet from around 3000 BC. Today, beer is brewed using various ingredients and methods, with over 100 different types of beer available.

Window memory:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about beer
AI: Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally ma

In [70]:
chat.run("What kinds of beer are there?")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about beer
AI: Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made by fermenting grains like barley, wheat, or rice. The first known recipe for beer was discovered in a Sumerian clay tablet from around 3000 BC. Today, beer is brewed using various ingredients and methods, with over 100 different types of beer available.
Human: What kinds of beer are there?
AI:

> Finished chain.
AI: There are over 100 different types of beer available, ranging from light lagers to dark stouts, from fruity ales to hoppy IPAs. Each type has its own unique flavor profile, color, 

In [71]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'simple', memory_window_size=2)
chat.run("Hello!")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:

Human: Hello!
AI:

> Finished chain.
AI: Hi there! How can I help you today?

Simple memory:
Human: Hello!
AI: Hi there! How can I help you today?


In [72]:
chat.run("Tell me some interesting facts about beer")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about beer
AI:

> Finished chain.
AI: Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made by fermenting wild grains like barley, wheat, or rice. Today, beer is brewed using a variety of ingredients and methods, with many different styles and flavors available. The process of making beer involves malting, mashing, boiling, cooling, fermentation, and conditioning. Beer contains various nutrients, including vitamins B6 and B12, folate, and potassium.

Simple memory:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about

Please make a short report abount differences between used memory types

In [73]:
chat.run("What kinds of beer are there?")
chat.print_memory()



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Hello!
AI: Hi there! How can I help you today?
Human: Tell me some interesting facts about beer
AI: Beer is the world's oldest alcoholic beverage, dating back to around 9500 BC. It was originally made by fermenting wild grains like barley, wheat, or rice. Today, beer is brewed using a variety of ingredients and methods, with many different styles and flavors available. The process of making beer involves malting, mashing, boiling, cooling, fermentation, and conditioning. Beer contains various nutrients, including vitamins B6 and B12, folate, and potassium.
Human: What kinds of beer are there?
AI:

> Finished chain.
AI: There are many different types of beer, categorized by their ingredients

Report:

1. `ConversationBufferMemory`: Данный тип памяти хранит весь диалог пользователя с LLM. В ходе диалога занимает все больше и больше памяти, так как добавляются новые сообщения. Подходит для задач, где важно сохранять полную историю диалога, чтобы модель могла учитывать все предшествующие сообщения.
2. `ConversationBufferWindowMemory`: Данный тип памяти хранит только заданное число взаимодействий пользователя с LLM, благодаря чему буфер имеет ограниченный размер. Подходит для задач, где используются краткосрочные диалоги.
3. `ConversationSummaryMemory`: Данный тип памяти создает краткий пересказ на основе взаимодействия человека с LLM. Подходит для ситуаций, когда полный контекст диалога не важен, но необходимо краткое описание предыдущих взаимодействий.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [74]:
openweather_key = os.environ.get("OPENWEATHER_API_KEY")

class WeatherReport(BaseModel):
    weather: str = Field(description="Краткое описание погоды в населенном пункте (ясно, дождь, пасмурно и т.д.)")
    temp: float = Field(description="Температура в населенном пункте")

few_shot_examples = [
    {
        "request": "Какая погода сейчас в Ставрополе?",
        "params": {"city": "Ставрополь"},
    }
]

@tool(few_shot_examples=few_shot_examples)
def get_weather(city: str) -> WeatherReport:
    """Returns weather and temperature in the provided city"""
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric")

    data = response.json()
    
    weather = data['weather'][0]['description'].capitalize()
    temp = data['main']['temp']
    
    return WeatherReport(weather=weather, temp=temp)

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.llm = llm
        self.agent_function = agent_function

        self.agent = create_gigachat_functions_agent(self.llm, [self.agent_function])

        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=[self.agent_function],
            verbose=True,
            )

    def run(self, user_input: str):
        if user_input == "":
            print("user_input must be non-empty!")
            return
        print(f"User: {user_input}")
        try:
            result = self.agent_executor.invoke(
                {
                    "input": user_input,
                }
            )
            print(f"Bot: {result['output']}")
        except Exception as e:
            print(f"Error: {str(e)}")


Let's check it

In [75]:

giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_weather)
user_input = "Какая погода сейчас в Сыктывкаре?"
openwheatertool.run(user_input)

User: Какая погода сейчас в Сыктывкаре?


> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Сыктывкар'}`


weather='Overcast clouds' temp=3.85В Сыктывкаре сейчас облачно. Температура воздуха составляет 3.9°С.

> Finished chain.
Bot: В Сыктывкаре сейчас облачно. Температура воздуха составляет 3.9°С.


In [76]:
openwheatertool.run("Какая погода сейчас в Лос-Анджелесе?")

User: Какая погода сейчас в Лос-Анджелесе?


> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Лос-Анджелес'}`


weather='Overcast clouds' temp=17.03Сейчас в Лос-Анджелесе облачно, температура 17.03°С.

> Finished chain.
Bot: Сейчас в Лос-Анджелесе облачно, температура 17.03°С.


## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [77]:
def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)

    if not r.ok:
        res = ''

    r_text = r.text
    r_json = r_text[5: len(r_text)-1]
    res_json = json.loads(r_json)
    res = res_json['results'][0]['geoid']

    return str(res)


In [78]:
class BookingHotelsReport(BaseModel):
    url: str = Field(description="Ссылка на сайт для бронирования отелей.")
    city: str = Field(description="Населенный пункт, в который планируется поездка.")
    date_in: datetime.datetime = Field(description="Предполагаемая дата заезда.")
    date_out: datetime.datetime = Field(description="Предполагаемая дата выезда.")

few_shot_examples = [
    {
        "request": "Организуй поездку в Санкт-Петербург на 6 дней с 21.10.2024 - отель, самолет, погода",
        "params": {"city": "Санкт-Петербург",
                   "date_in_str": "21.10.2024",
                   "date_out_str": "27.10.2024"},
    }
]

@tool(few_shot_examples=few_shot_examples)
def get_url_booking_hotels(date_in_str: str,
                           date_out_str: str,
                           city: str) -> BookingHotelsReport:
    """Returns the url for booking a hotel in the provided city on the provided dates"""

    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)

    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)

    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?'

    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    
    return BookingHotelsReport(url=url,
                               city=city,
                               date_in=date_in,
                               date_out=date_out)


In [79]:
class BookingTicketsReport(BaseModel):
    url: str = Field(description="Ссылка на сайт для бронирования билетов.")
    city_from: str = Field(description="Населенный пункт, из которого планируется поездка.")
    city_to: str = Field(description="Населенный пункт, в который планируется поездка.")
    date_in: datetime.datetime = Field(description="Предполагаемая дата отправления в путешествие.")
    date_out: datetime.datetime = Field(description="Предполагаемая дата отправления обратно.")

few_shot_examples = [
    {
        "request": "Организуй поездку в Санкт-Петербург на 6 дней с 21.10.2024 - отель, самолет, погода",
        "params": {"city_from":
                   "Москва", "city_to":
                   "Санкт-Петербург",
                   "date_in_str": "21.10.2024",
                   "date_out_str": "27.10.2024"},
    }
]

@tool(few_shot_examples=few_shot_examples)
def get_url_booking_tikets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> BookingTicketsReport:
    """Returns the url for booking tickets between the provided cities on the provided dates"""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?' 
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    return BookingTicketsReport(url=url,
                                 city_from=city_from,
                                 city_to=city_to,
                                 date_in=date_in,
                                 date_out=date_out)


In [80]:
class MultiAgent:
    def __init__(self,
                 llm,
                 agent_function_weather,
                 agent_function_hotels,
                 agent_function_tickets):
        self.llm = llm

        self.agent_function_weather = agent_function_weather
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

        self.check_weather_agent_executor = self._create_agent([self.agent_function_weather],
                                                               "You are a bot for answering questions about the weather.")
        self.hotels_booking_agent_executor = self._create_agent([self.agent_function_hotels],
                                                                "You are a bot that helps you book hotels.")
        self.tickets_booking_agent_executor = self._create_agent([self.agent_function_tickets],
                                                                "You are a bot that helps you book tickets.")

    def _create_agent(self, tools, system_prompt):
        agent = create_gigachat_functions_agent(self.llm, tools)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def run(self, user_input: str):
        weather_info = self.check_weather_agent_executor.invoke({"input": f"Сообщи погоду в городе назначения по информации из {user_input}?"})
        hotels_info = self.hotels_booking_agent_executor.invoke({"input": f"Предоставь информацию об отелях в городе назначения по информации из {user_input}?"})
        tickets_info = self.tickets_booking_agent_executor.invoke({"input": f"Предоставь информацию о билетах между городами по информации из {user_input}?"})

        return f"Информация об отелях: {hotels_info['output']}\nИнформация о билетах: {tickets_info['output']}\nИнформация о погоде: {weather_info['output']}"


In [81]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)



> Entering new AgentExecutor chain...



Invoking: `get_weather` with `{'city': 'Санкт-Петербург'}`


weather='Broken clouds' temp=9.47В Санкт-Петербурге ожидается облачная погода с прояснениями. Температура воздуха составит около 9.47°С.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_url_booking_hotels` with `{'city': 'Санкт-Петербург', 'date_in_str': '15.11.2024', 'date_out_str': '24.11.2024'}`


url='https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2' city='Санкт-Петербург' date_in=datetime.datetime(2024, 11, 15, 0, 0) date_out=datetime.datetime(2024, 11, 24, 0, 0)Я нашел для Вас информацию об отелях в Санкт-Петербурге на указанные даты. Вы можете посмотреть доступные варианты на сайте https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-24&childrenAges=0&geoId=2.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_url_booking_tikets` with `{'city_from': 'Москва',

In [82]:
user_input = "Организуй поездку из Санкт-Петерубрга в Сыктывкар на 10 дней с 21.10.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Сыктывкар'}`


weather='Overcast clouds' temp=3.85В Сыктывкаре ожидается облачная погода. Температура воздуха составит 3.85°С.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_url_booking_hotels` with `{'city': 'Сыктывкар', 'date_in_str': '21.10.2024', 'date_out_str': '30.10.2024'}`


url='https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-10-21&checkoutDate=2024-10-30&childrenAges=0&geoId=19' city='Сыктывкар' date_in=datetime.datetime(2024, 10, 21, 0, 0) date_out=datetime.datetime(2024, 10, 30, 0, 0)Я нашёл для Вас информацию об отелях в Сыктывкаре на указанные даты. Вы можете забронировать номер, перейдя по этой ссылке: [travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-10-21&checkoutDate=2024-10-30&childrenAges=0&geoId=19](https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-10-21&checkoutDate=2024-10-30&childrenAges=0&geoId=19).

> Finish